# Data Gathering and Preparation

In [1]:
# Import the requisite packages
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from census import Census 
from us import states 

In [2]:
# Set API key
c = Census("2ae6b4b06c6b461ac27efd364c2fa133d96c0051")

In [16]:
# Grabbing a few variables of interest pertaining to the economic vitality of an area
fields = [
    "NAME",
    "B01003_001E", #"Total Population"
    "B25077_001E", #"Median value of owner occupied units"
    "B25026_001E", #"Total population in occupied housing units"
    "B25008_002E", #"Total number of owner occupied units"
    "B25008_003E", #"Total number of renter occupied units"
    "B06009_002E", #"Population with less than a high school diploma"
    "B06009_003E", #"Population with high school diploma or equivalent"
    "B06009_004E", #"Population with some college/associates degree"
    "B06009_005E", #"Population with bachelors degree"
    "B06009_006E", #"Population with a graduate degree"
    "B01002_001E", #"Median age"
    "B06010_004E", #"Population with income less than 9999"
    "B06010_005E", #"Population with income between 10000 and 14999"
    "B06010_006E", #"Population with income between 15000 and 24999"
    "B06010_007E", #"Population with income between 25000 and 34999"
    "B06010_008E", #"Population with income between 35000 and 49999"
    "B06010_009E", #"Population with income between 50000 and 64999"
    "B06010_010E", #"Population with income between 65000 and 74999"
    "B06010_011E", #"Population with income of 75000 or more"
    "B28007_009E", #"Population in labor force and unemployed"
    "B19059_002E", #"Population that is retired with retirement income"
    "B19059_003E", #"Retired without retirement income"
    "B08013_001E", #"Travel time to work in minutes"
    "B17013_002E" #"Population with income below poverty level in past 12 months"
        ]

In [17]:
# Sources: https://api.census.gov/data/2019/acs/acs5/variables.html; https://pypi.org/project/census/
nm_census = c.acs5.state_county_tract(
    fields = fields,
    state_fips = states.NM.fips,
    county_fips = "*",
    tract = "*",
    year = 2022
    )

In [42]:
nm_df = pd.DataFrame(nm_census)
print(nm_df.head(2))

                                               NAME  B01003_001E  B25077_001E  \
0  Census Tract 1.07; Bernalillo County; New Mexico       2535.0     423200.0   
1  Census Tract 1.08; Bernalillo County; New Mexico       2539.0     323900.0   

   B25026_001E  B25008_002E  B25008_003E  B06009_002E  B06009_003E  \
0       2529.0       2111.0        418.0          0.0        239.0   
1       2509.0       1539.0        970.0        118.0        223.0   

   B06009_004E  B06009_005E  ...  B06010_010E  B06010_011E  B28007_009E  \
0        466.0        753.0  ...         86.0        636.0         33.0   
1        806.0        514.0  ...        178.0        481.0         30.0   

   B19059_002E  B19059_003E  B08013_001E  B17013_002E  state  county   tract  
0        370.0        834.0      22245.0         65.0     35     001  000107  
1        232.0       1071.0      22730.0         34.0     35     001  000108  

[2 rows x 28 columns]


In [43]:
nm_df.shape

(612, 28)

> NM fips: 35
> EPSG: 2258

In [44]:
nm_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2022/TRACT/tl_2022_35_tract.zip")

In [45]:
nm_tract = nm_tract.to_crs(2258)

In [46]:
nm_tract.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,35,009,000308,35009000308,3.08,Census Tract 3.08,G5020,S,5241119,40343,+34.4271346,-103.1803059,"POLYGON ((2561417.241 1257878.087, 2561398.206..."
1,35,009,000307,35009000307,3.07,Census Tract 3.07,G5020,S,2640449,12504,+34.4408610,-103.1884611,"POLYGON ((2561269.483 1263468.297, 2561258.319..."


In [22]:
nm_tract.shape

(612, 13)

In [66]:
nm_tract.loc[nm_tract.GEOID.str.contains('105')].GEOID

147    35049001105
239    35013001105
585    35043010503
594    35049010500
Name: GEOID, dtype: object

In [67]:
print(nm_df.loc[nm_df.tract.str.contains('105')])

                                                 NAME  B01003_001E  \
241   Census Tract 11.05; Doña Ana County; New Mexico       2319.0   
423  Census Tract 105.03; Sandoval County; New Mexico       3538.0   
519   Census Tract 11.05; Santa Fe County; New Mexico       2609.0   
546     Census Tract 105; Santa Fe County; New Mexico       1994.0   

     B25077_001E  B25026_001E  B25008_002E  B25008_003E  B06009_002E  \
241 -666666666.0       2319.0       1364.0        955.0        329.0   
423     171300.0       3538.0       2506.0       1032.0        538.0   
519     282800.0       2603.0       1682.0        921.0         49.0   
546     845000.0       1994.0       1564.0        430.0         45.0   

     B06009_003E  B06009_004E  B06009_005E  ...  B06010_010E  B06010_011E  \
241        470.0        308.0        166.0  ...          0.0        138.0   
423       1008.0        854.0        392.0  ...        200.0        357.0   
519        381.0        712.0        462.0  ...        25

>Create GEOID variable to merge with census data

In [68]:
nm_df['GEOID'] = nm_df.state + nm_df.county + nm_df.tract

In [69]:
nm_df = nm_df.drop(columns=['state', 'county', 'tract'])

In [70]:
nm_merge = nm_tract.merge(nm_df, on='GEOID')
print(nm_merge.head(2))

  STATEFP COUNTYFP TRACTCE        GEOID NAME_x           NAMELSAD  MTFCC  \
0      35      009  000308  35009000308   3.08  Census Tract 3.08  G5020   
1      35      009  000307  35009000307   3.07  Census Tract 3.07  G5020   

  FUNCSTAT    ALAND  AWATER  ... B06010_007E B06010_008E B06010_009E  \
0        S  5241119   40343  ...       144.0       434.0       326.0   
1        S  2640449   12504  ...       347.0       505.0       292.0   

  B06010_010E  B06010_011E  B28007_009E  B19059_002E  B19059_003E  \
0       174.0        477.0        130.0        460.0        759.0   
1       117.0        637.0         96.0        299.0       1311.0   

   B08013_001E  B17013_002E  
0      23240.0         19.0  
1      33020.0         41.0  

[2 rows x 38 columns]


In [71]:
nm_merge.shape

(612, 38)

In [72]:
# Renaming variables in the data set
nm_merge.rename(columns={
    "B01003_001E":"TotPop", #"Total Population"
    "B25077_001E":"MedVal_OwnOccUnit", #"Median value of owner occupied units"
    "B25026_001E":"TotPopOccUnits", #"Total population in occupied housing units"
    "B25008_002E":"TotNumOwnOccUnit", #"Total number of owner occupied units"
    "B25008_003E":"TotNumRentOccUnit", #"Total number of renter occupied units"
    "B06009_002E":"PopLTHSDip", #"Population with less than a high school diploma"
    "B06009_003E":"PopHSDip", #"Population with high school diploma or equivalent"
    "B06009_004E":"PopAssoc", #"Population with some college/associates degree"
    "B06009_005E":"PopBA", #"Population with bachelors degree"
    "B06009_006E":"PopGrad", #"Population with a graduate degree"
    "B01002_001E":"MedAge", #"Median age"
    "B06010_004E":"PopIncLT10", #"Population with income less than 9999"
    "B06010_005E":"PopInc1015", #"Population with income between 10000 and 14999"
    "B06010_006E":"PopInc1525", #"Population with income between 15000 and 24999"
    "B06010_007E":"PopInc2535", #"Population with income between 25000 and 34999"
    "B06010_008E":"PopInc3550", #"Population with income between 35000 and 49999"
    "B06010_009E":"PopInc5065", #"Population with income between 50000 and 64999"
    "B06010_010E":"PopInc6575", #"Population with income between 65000 and 74999"
    "B06010_011E":"PopIncGT75", #"Population with income of 75000 or more"
    "B28007_009E":"UnempPop", #"Population in labor force and unemployed"
    "B19059_002E":"RetPop", #"Population that is retired with retirement income"
    "B19059_003E":"RetPopNoRetInc", #"Retired without retirement income"
    "B08013_001E":"TrvTimWrk", #"Travel time to work in minutes"
    "B17013_002E":"PopBlwPovLvl" #"Population with income below poverty level in past 12 months"
}
                , inplace=True
               )

>Create a subset of variables of interest

In [73]:
geo_demo_rn = [
"TotPop", #"Total Population"
"TotPopOccUnits", #"Total population in occupied housing units"
"TotNumOwnOccUnit", #"Total number of owner occupied units"
"TotNumRentOccUnit", #"Total number of renter occupied units"
"PopLTHSDip", #"Population with less than a high school diploma"
"PopHSDip", #"Population with high school diploma or equivalent"
"PopAssoc", #"Population with some college/associates degree"
"PopBA", #"Population with bachelors degree"
"PopGrad", #"Population with a graduate degree"
"PopIncLT10", #"Population with income less than 9999"
"PopInc1015", #"Population with income between 10000 and 14999"
"PopInc1525", #"Population with income between 15000 and 24999"
"PopInc2535", #"Population with income between 25000 and 34999"
"PopInc3550", #"Population with income between 35000 and 49999"
"PopInc5065", #"Population with income between 50000 and 64999"
"PopInc6575", #"Population with income between 65000 and 74999"
"PopIncGT75", #"Population with income of 75000 or more"
"UnempPop", #"Population in labor force and unemployed"
"RetPop", #"Population that is retired with retirement income"
"RetPopNoRetInc", #"Retired without retirement income"
"PopBlwPovLvl" #"Population with income below poverty level in past 12 months"
]

In [74]:
geo_demo_rn.append('geometry')

In [75]:
nm_merge2 = nm_merge[geo_demo_rn]

In [76]:
geo_demo_rn.remove('geometry')

>Remove areas with no population

In [77]:
nm_merge2.shape

(612, 22)

In [78]:
nm_merge2 = nm_merge2[nm_merge2.TotPop > 0]
nm_merge2.reset_index(inplace=True)
nm_merge2.shape

(604, 23)

In [79]:
nm_merge2.to_file('../data/nm_merge2.gpkg')

In [80]:
nm_tract.to_file('../data/nm_tract.gpkg')

In [81]:
nm_merge.to_file('../data/nm_merge.gpkg')